In [11]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 이미지 로드 및 전처리 함수
def load_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image

# 사전 훈련된 모델을 사용하여 특징 추출 함수
def extract_features(model, image):
    with torch.no_grad():
        features = model(image)
    return features.view(features.size(0), -1).cpu().numpy()

# 사전 훈련된 모델 로드
model = models.resnet50(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])
model.eval()

# 이미지 전처리 변환
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 이미지 디렉토리 경로
japan_dir = 'japan'  # 일본 이미지 경로
korea_dir = 'korea'  # 한국 이미지 경로

# 모든 이미지 로드
japan_images = [os.path.join(japan_dir, img) for img in os.listdir(japan_dir) if img.endswith(('jpg', 'png'))]
korea_images = [os.path.join(korea_dir, img) for img in os.listdir(korea_dir) if img.endswith(('jpg', 'png'))]

# 한국 이미지의 모든 특징 추출
korea_features = []
for img_path in korea_images:
    img = load_image(img_path, transform)
    features = extract_features(model, img)
    korea_features.append(features)

korea_features = np.array(korea_features).squeeze()

# 각 일본 이미지에 대해 한국 이미지 중 가장 유사한 상위 3개 이미지 찾기
results = {}
for i, japan_img_path in enumerate(japan_images):
    japan_img = load_image(japan_img_path, transform)
    japan_features = extract_features(model, japan_img).squeeze()
    
    similarities = cosine_similarity([japan_features], korea_features)[0]
    top3_indices = similarities.argsort()[-1:][::-1]  # 100개의 한국 지역 중 top 50
    results[japan_img_path] = [(korea_images[idx], similarities[idx]) for idx in top3_indices]

# 결과 출력
for japan_img, similar_imgs in results.items():
    print(f"Japan Image: {japan_img}")
    for img, sim in similar_imgs:
        print(f"  Similar Korea Image: {img} with similarity {sim}")

/Users/ilhudy/miniconda3/envs/vision39/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ilhudy/miniconda3/envs/vision39/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Japan Image: japan/ALL_Tsukiji Outer Market_35.6655_139.7708.png
  Similar Korea Image: korea/ALL_Sacheon-si_35.0497034130916_128.037696348307.png with similarity 0.7649810314178467
Japan Image: japan/ALL_Roppongi Hills_35.6604_139.7292.png
  Similar Korea Image: korea/ALL_Gimje-si_35.8067164440207_126.894885695778.png with similarity 0.7872594594955444
Japan Image: japan/ALL_Osaka Castle_34.6873_135.5259.png
  Similar Korea Image: korea/ALL_Yesan-gun_36.6706286680345_126.784310944751.png with similarity 0.7449114322662354
Japan Image: japan/ALL_Nikko Toshogu Shrine_36.7578_139.5983.png
  Similar Korea Image: korea/ALL_Jinhae-gu, Changwon-si_35.1299494537177_128.736300826452.png with similarity 0.8781305551528931
Japan Image: japan/ALL_Kiyomizu-dera Temple_34.9949_135.785.png
  Similar Korea Image: korea/ALL_Yesan-gun_36.6706286680345_126.784310944751.png with similarity 0.6667792797088623
Japan Image: japan/ALL_Miyajima_34.2956_132.3198.png
  Similar Korea Image: korea/ALL_Wando-gun_3

In [14]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 이미지 로드 및 전처리 함수
def load_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image

# 사전 훈련된 모델을 사용하여 특징 추출 함수
def extract_features(model, image):
    with torch.no_grad():
        features = model(image)
    return features.view(features.size(0), -1).cpu().numpy()

# 사전 훈련된 모델 로드
model = models.resnet50(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])
model.eval()

# 이미지 전처리 변환
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 이미지 디렉토리 경로
japan_dir = 'japan'  # 일본 이미지 경로
korea_dir = 'korea'  # 한국 이미지 경로

# 모든 이미지 로드
japan_images = [os.path.join(japan_dir, img) for img in os.listdir(japan_dir) if img.endswith(('jpg', 'png'))]
korea_images = [os.path.join(korea_dir, img) for img in os.listdir(korea_dir) if img.endswith(('jpg', 'png'))]

# 한국 이미지의 모든 특징 추출
korea_features = []
for img_path in korea_images:
    img = load_image(img_path, transform)
    features = extract_features(model, img)
    korea_features.append(features)

korea_features = np.array(korea_features).squeeze()

# 각 일본 이미지에 대해 한국 이미지 중 가장 유사한 상위 3개 이미지 찾기
results = []
for i, japan_img_path in enumerate(japan_images):
    japan_img = load_image(japan_img_path, transform)
    japan_features = extract_features(model, japan_img).squeeze()
    
    similarities = cosine_similarity([japan_features], korea_features)[0]
    top3_indices = similarities.argsort()[-3:][::-1]
    for idx in top3_indices:
        results.append({
            'Japan Image': japan_img_path,
            'Korea Image': korea_images[idx],
            'Similarity': similarities[idx]
        })

# 결과를 데이터프레임으로 변환하고 CSV 파일로 저장
df = pd.DataFrame(results)
df.to_csv('image_similarity_results.csv', index=False)

print("CSV 파일이 저장되었습니다.")

/Users/ilhudy/miniconda3/envs/vision39/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ilhudy/miniconda3/envs/vision39/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CSV 파일이 저장되었습니다.


In [17]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 이미지 로드 및 전처리 함수
def load_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image

# 사전 훈련된 모델을 사용하여 특징 추출 함수
def extract_features(model, image):
    with torch.no_grad():
        features = model(image)
    return features.view(features.size(0), -1).cpu().numpy()

# 사전 훈련된 모델 로드
model = models.resnet50(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])
model.eval()

# 이미지 전처리 변환
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 이미지 디렉토리 경로
japan_dir = 'japan'  # 일본 이미지 경로
korea_dir = 'korea'  # 한국 이미지 경로

# 모든 이미지 로드
japan_images = [os.path.join(japan_dir, img) for img in os.listdir(japan_dir) if img.endswith(('jpg', 'png'))]
korea_images = [os.path.join(korea_dir, img) for img in os.listdir(korea_dir) if img.endswith(('jpg', 'png'))]

# 한국 이미지의 모든 특징 추출
korea_features = []
for img_path in korea_images:
    img = load_image(img_path, transform)
    features = extract_features(model, img)
    korea_features.append(features)

korea_features = np.array(korea_features).squeeze()

# 각 일본 이미지에 대해 한국 이미지 중 가장 유사한 상위 50개 이미지 찾기
results = []
for i, japan_img_path in enumerate(japan_images):
    japan_img = load_image(japan_img_path, transform)
    japan_features = extract_features(model, japan_img).squeeze()
    
    similarities = cosine_similarity([japan_features], korea_features)[0]
    top3_indices = similarities.argsort()[-50:][::-1]
    for idx in top3_indices:
        if similarities[idx] >= 0.7:
            results.append({
                'Japan Image': japan_img_path,
                'Korea Image': korea_images[idx],
                'Similarity': similarities[idx]
            })

# 결과를 데이터프레임으로 변환하고 CSV 파일로 저장
df = pd.DataFrame(results)
df.to_csv('image_similarity_results_0.7.csv', index=False)

print("CSV 파일이 저장되었습니다.")

/Users/ilhudy/miniconda3/envs/vision39/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ilhudy/miniconda3/envs/vision39/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CSV 파일이 저장되었습니다.
